<a href="https://colab.research.google.com/github/clementsiegrist/image_editing/blob/main/image_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mosaic

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/Datadolittle/Photo_Mosaic.git
%cd Photo_Mosaic/

In [ ]:
!python scripts/Mosaic_Creator.py --target /content/drive/MyDrive/best/IMG_20220320_164103.jpg --images /content/drive/MyDrive/best/ --grid 200 200 --output data/Mosaic.jpeg

# Background substraction and removal

In [ ]:
%cd ../

In [ ]:
!git clone https://github.com/thiagoambiel/PortraitStylization.git
%cd PortraitStylization/
!pip install -r requirements.txt

In [ ]:
from PIL import Image
from style_transfer import StyleTransfer
from remove_bg import BackgroundRemoval

# Load content image.
original_image = Image.open("content.jpg")

# Load MODNet and remove content image background.
background_removal = BackgroundRemoval("./weights/modnet.pth", device="cpu")

content_image = background_removal.remove_background(
    img=original_image,
    bg_color="black",
)

# Load style images.
style_images = [
    Image.open("style_1.jpg"), 
    Image.open("style_2.jpg"),
]

# Load and run style transfer module.
st = StyleTransfer(device="cpu", pooling="max")

result_image = st.stylize(
    content_image=content_image, 
    style_images=style_images,
    content_weight=0.05,
    face_weight=0.25,
)

# Save result to disk.
result_image.save("out.png")

## Style Transfer 

In [ ]:
!python stylize.py input.jpg  -o out.png -cw 0.05 -fw 0.25

## Background removal 

# Image Concat

https://note.nkmk.me/en/python-pillow-concat-images/

In [ ]:
def get_concat_h(im1, im2):

    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v(im1, im2):

    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def get_concat_h_cut(im1, im2):

    dst = Image.new('RGB', (im1.width + im2.width, min(im1.height, im2.height)))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v_cut(im1, im2):

    dst = Image.new('RGB', (min(im1.width, im2.width), im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def get_concat_h_cut_center(im1, im2):

    dst = Image.new('RGB', (im1.width + im2.width, min(im1.height, im2.height)))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, (im1.height - im2.height) // 2))
    return dst

def get_concat_v_cut_center(im1, im2):

    dst = Image.new('RGB', (min(im1.width, im2.width), im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, ((im1.width - im2.width) // 2, im1.height))
    return dst

def get_concat_h_blank(im1, im2, color=(0, 0, 0)):

    dst = Image.new('RGB', (im1.width + im2.width, max(im1.height, im2.height)), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v_blank(im1, im2, color=(0, 0, 0)):

    dst = Image.new('RGB', (max(im1.width, im2.width), im1.height + im2.height), color)
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def get_concat_h_multi_resize(im_list, resample=Image.BICUBIC):

    min_height = min(im.height for im in im_list)
    im_list_resize = [im.resize((int(im.width * min_height / im.height), min_height),resample=resample)
                      for im in im_list]
    total_width = sum(im.width for im in im_list_resize)
    dst = Image.new('RGB', (total_width, min_height))
    pos_x = 0
    for im in im_list_resize:
        dst.paste(im, (pos_x, 0))
        pos_x += im.width
    return dst

def get_concat_v_multi_resize(im_list, resample=Image.BICUBIC):

    min_width = min(im.width for im in im_list)
    im_list_resize = [im.resize((min_width, int(im.height * min_width / im.width)),resample=resample)
                      for im in im_list]
    total_height = sum(im.height for im in im_list_resize)
    dst = Image.new('RGB', (min_width, total_height))
    pos_y = 0
    for im in im_list_resize:
        dst.paste(im, (0, pos_y))
        pos_y += im.height

    return dst

def get_concat_h_repeat(im, column):

    dst = Image.new('RGB', (im.width * column, im.height))
    for x in range(column):
        dst.paste(im, (x * im.width, 0))
    return dst

def get_concat_v_repeat(im, row):

    dst = Image.new('RGB', (im.width, im.height * row))
    for y in range(row):
        dst.paste(im, (0, y * im.height))
    return dst

def get_concat_tile_repeat(im, row, column):

    dst_h = get_concat_h_repeat(im, column)
    return get_concat_v_repeat(dst_h, row)

def get_concat_tile_resize(im_list_2d, resample=Image.BICUBIC):

    im_list_v = [get_concat_h_multi_resize(im_list_h, resample=resample) for im_list_h in im_list_2d]
    return get_concat_v_multi_resize(im_list_v, resample=resample)

In [ ]:
from PIL import Image

im1 = Image.open('/content/drive/MyDrive/best/IMG_20220320_164103.jpg')
im2 = Image.open('/content/drive/MyDrive/best/charo_nouer.png')
im3 = Image.open('/content/drive/MyDrive/best/202203114774371443845825173.jpg')
im4 = Image.open('/content/drive/MyDrive/best/20220220_152309.jpg')

In [ ]:
get_concat_h(im1, im1).save('data/dst/pillow_concat_h.jpg')
get_concat_v(im1, im1).save('data/dst/pillow_concat_v.jpg')

get_concat_h_cut(im1, im2).save('data/dst/pillow_concat_h_cut.jpg')
get_concat_v_cut(im1, im2).save('data/dst/pillow_concat_v_cut.jpg')

get_concat_h_cut_center(im1, im2).save('data/dst/pillow_concat_h_cut_center.jpg')
get_concat_v_cut_center(im1, im2).save('data/dst/pillow_concat_v_cut_center.jpg')

get_concat_h_multi_resize([im1, im2, im1]).save('data/dst/pillow_concat_h_multi_resize.jpg')
get_concat_v_multi_resize([im1, im2, im1]).save('data/dst/pillow_concat_v_multi_resize.jpg')

In [ ]:
get_concat_tile_resize([[im1],
                        [im1, im2],
                        [im3, im1, im2],
                        [im1, im2, im3, im4]]).save('pillow_concat_tile_resize.jpg')

In [ ]:
get_concat_tile_resize([[im2],
                        [im1, im2],
                        [im3, im1, im4]]).save('pillow_concat_tile_resize_3.jpg')